In [1]:
import os
import sys

# change to upper dir to use all custom libs (won't be needed if run from main scripts)
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [2]:
from scripts.helpers.model_manager import NLLB200Model

model = NLLB200Model(device='cuda')

model.translate_batch(['Счастливые лисички прыгают прыг скок', 'раз два три!'], 'ru', 'kaz')

['Счастливые лисички прыгают прыгают скак Бақытты лисички секіріп секіріп секіреді',
 'Бір, екі, үш рет!']

In [3]:
from scripts.helpers.dataset_manager import HFDataset
import pandas as pd

tt_ru = HFDataset('tat-ru', 'tatar-parallel-162k').get_df()

In [4]:
model._train(
    data=tt_ru,
    src_lang='tat',
    dst_lang='rus',
    training_steps=100,
    save_every=20,
    ttl=10,
)

  0%|          | 0/100 [00:00<?, ?it/s]

0 1.693359375
Triggered time to leave! Current time: 1739916397.6928473. Start time: 1739916372.5352678. TTL: 10.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\cutefluffyfox\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\serializat │
│ ion.py:619 in save                                                                               │
│                                                                                                  │
│    616 │                                                                                         │
│    617 │   if _use_new_zipfile_serialization:                                                    │
│    618 │   │   with _open_zipfile_writer(f) as opened_zipfile:                                   │
│ ❱  619 │   │   │   _save(obj, opened_zipfile, pickle_module, pickle_protocol, _disable_byteorde  │
│    620 │   │   │   return                                                                        │
│    621 │   else:                                                                                 │
│    622 │   │   with _open_file_like(f, 'wb') as opened_file:                                     │
│                                                                                                  │
│ C:\Users\cutefluffyfox\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\serializat │
│ ion.py:853 in _save                                                                              │
│                                                                                                  │
│    850 │   │   │   storage = storage.cpu()                                                       │
│    851 │   │   # Now that it is on the CPU we can directly copy it into the zip file             │
│    852 │   │   num_bytes = storage.nbytes()                                                      │
│ ❱  853 │   │   zip_file.write_record(name, storage.data_ptr(), num_bytes)                        │
│    854                                                                                           │
│    855                                                                                           │
│    856 def load(                                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: [enforce fail at inline_container.cc:593] . PytorchStreamWriter failed writing file data/365: file 
write failed

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model._train(                                                                                │
│   2 │   data=tt_ru,                                                                              │
│   3 │   src_lang='tat',                                                                          │
│   4 │   dst_lang='rus',                                                                          │
│                                                                                                  │
│ C:\Users\cutefluffyfox\PycharmProjects\MT-vocab-tuning\notebooks\..\scripts\helpers\model_manage │
│ r.py:172 in _train                                                                               │
│                                                                                                  │
│   169 │   │   │   │   self.tokenizer.save_pretrained(f"tokenizer.sft.{src_lang}_{dst_lang}.{i}   │
│   170 │   │                                                                                      │
│   171 │   │   # save final model                                                                 │
│ ❱ 172 │   │   self.model.save_pretrained(f"model.sft.{src_lang}_

In [21]:
import yadisk
import posixpath

def recursive_upload(client: yadisk.Client, from_dir: str, to_dir: str):
    for root, dirs, files in os.walk(from_dir):
        p = root.split(from_dir)[1].strip(os.path.sep)
        dir_path = posixpath.join(to_dir, p)

        try:
            client.mkdir(dir_path)
        except yadisk.exceptions.PathExistsError:
            pass

        for file in files:
            file_path = posixpath.join(dir_path, file)
            p_sys = p.replace("/", os.path.sep)
            in_path = os.path.join(from_dir, p_sys, file)

            try:
                client.upload(in_path, file_path)
            except yadisk.exceptions.PathExistsError:
                pass

In [22]:
client = yadisk.Client(token="y0_AgAAAAAqH3aPAAyjSAAAAAEVKiGwAAB04VoQH5dMj6QavFOfkBLr4zidNA")

# You can either use the with statement or manually call client.close() later
with client:
    # Check if the token is valid
    print(client.check_token())

    # Print files and directories at "/some/path"
    print(list(client.listdir("/bs-diploma/experiments")))
    
    # Upload model checkpoint
#     client.mkdir("/bs-diploma/experiments/local-sft-run", exists_ok=True)
#     client.mkdir("/bs-diploma/experiments/local-sft-run/tokenizer.sft.tat_rus.80", exists_ok=True)
    
    recursive_upload(client, 'tokenizer.sft.tat_rus.80', "/bs-diploma/experiments/local-sft-run/tokenizer.sft.tat_rus.80")

True
[<SyncResourceObject{'antivirus_status': None, 'file': None, 'size': None, 'public_key': None, 'sha256': None, 'embedded': None, 'name': 'local-sft-run', 'exif': <EXIFObject{'date_time': None, 'gps_longitude': None, 'gps_latitude': None}>, 'resource_id': '706705039:b4eea1744be73cbd31a92397015164504c3a45756da6c4fad5c3d448db663f50', 'custom_properties': None, 'public_url': None, 'share': None, 'modified': datetime.datetime(2025, 2, 18, 22, 16, 12, tzinfo=datetime.timezone.utc), 'created': datetime.datetime(2025, 2, 18, 22, 16, 12, tzinfo=datetime.timezone.utc), 'photoslice_time': None, 'mime_type': None, 'path': 'disk:/bs-diploma/experiments/local-sft-run', 'preview': None, 'comment_ids': <CommentIDsObject{'private_resource': '706705039:b4eea1744be73cbd31a92397015164504c3a45756da6c4fad5c3d448db663f50', 'public_resource': '706705039:b4eea1744be73cbd31a92397015164504c3a45756da6c4fad5c3d448db663f50'}>, 'type': 'dir', 'media_type': None, 'md5': None, 'revision': 1739916972153830}>]


In [24]:
a = 1
if a is not None:
    print('hello')

hello
